In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np
import pandas_profiling

# 분류문제를 위한 모델 import
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 분류모델 평가를 위한 accuracy_score import
from sklearn.metrics import accuracy_score
# 분류모델 평가를 위한 train/test set 분리를 위한 train_test_split import
from sklearn.model_selection import train_test_split
# 정규화를 위한 StandardScaler import
from sklearn.preprocessing import StandardScaler

# Seed 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [2]:
# 분류 모델 평가를 위해 default값을 넣은 분류모델을 각각 학습
# train 과 test으로 분리한 데이터로 각 분류모델별 accuracy_score 계산 후 result 배열에 넣기
def _classification(x, y, result:list):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    
    for model in [KNeighborsClassifier(), KNeighborsClassifier(weights='distance'), LogisticRegression(solver='newton-cg'), LinearDiscriminantAnalysis(), DecisionTreeClassifier(), RandomForestClassifier()]:
        model.fit(x_train, y_train)
        res = model.predict(x_test)
        result.append({'model': str(model), 'score': accuracy_score(res, y_test)})

In [3]:
# 데이터 불러오기
data = pd.read_csv('../input/2022-ml-midterm-p1/train.csv').drop('index', axis=1)
test = pd.read_csv('../input/2022-ml-midterm-p1/test.csv')

_data = data

# Outlier 제거
data = data.drop(data[(data.thal == -1) | (data.ca == -1)].index)

# data.loc[data.target > 1, 'target'] = 1
data.loc[data.restecg == 2, 'restecg'] = 1

x = data.drop(['target', 'fbs'], axis=1)
test = test.drop(['index', 'fbs'], axis=1)
y = data['target']


In [4]:
sc = StandardScaler()
x = sc.fit_transform(x, y)
test = sc.transform(test)

In [5]:
result = []
_classification(x, y, result)
print(pd.DataFrame(result))

                                      model     score
0                    KNeighborsClassifier()  0.541667
1  KNeighborsClassifier(weights='distance')  0.520833
2    LogisticRegression(solver='newton-cg')  0.645833
3              LinearDiscriminantAnalysis()  0.645833
4                  DecisionTreeClassifier()  0.437500
5                  RandomForestClassifier()  0.645833


In [6]:
submit = pd.read_csv('../input/2022-ml-midterm-p1/submit.csv')
model = LogisticRegression(solver='newton-cg')
model.fit(x, y)
res = model.predict(test)
submit['target'] = res
submit.loc[submit.target > 1, 'target'] = 1
submit.to_csv('submission.csv', index=False)
submit

,index,target
0,3,0
1,6,1
2,21,0
3,24,1
4,31,1
...,...,...
56,293,1
57,294,0
58,297,0
59,298,0
